:::{index} single: Impact Decomposition 
:::


In [ ]:
#This is code to manage dependencies if the notebook is executed in the google colab cloud service
if 'google.colab' in str(get_ipython()):
  import os
  os.system('apt -qqq install graphviz')
  os.system('pip -qqq install ModelFlowIb   ')


In [ ]:
%load_ext autoreload
%autoreload 2   



# Analyzing the impact of a shock 

When working with a model, it is often useful to have a quantitative of the contribution of different channels to a final result.  For example, an increase in interest rates will tend to reduce investment and consumer demand -- contributing to a reduction in GDP. At the same time, lower inflation as the higher interest rate takes effect will tend to work in the opposite direction. 

The `tracedep()` and `tracepre()` methods introduced in the previous chapter give a sense of impacts. The `ModelFlow` methods `.dekomp()` and `.totdif()` take that one step further by calculating the contribution of each channel to the overall result.



:::{admonition} In this chapter - Analyzing the Impact of a Shock
:class: tip

This chapter provides the tools and techniques to dissect and understand the ripple effects of shocks within macroeconomic systems.

This chapter focuses on methods to analyze the impacts of external shocks within a macroeconomic model. The previous chapter explored the impacts of changes in one variable on other variables in a model by following the causal chain of any individual variable.

The techniques explored in this chapter illustrate and extend this by:
- attributing changes in any given model variable to the changes in the variables in the model that were shocked.  In the techniques used in the previous chapter it may be determined that increased inflation was the proximate cause of a decline in consumption, the methods presented here seek to illuminate which of the changes to exogenous variables caused the increase in inflation that caused the decline in consumption (say an increase in oil prices).
- Impacts can be traced either through a single equation method or using a model level decomposition

Examples include a range of graphical and textual visualization of results.

:::


In [ ]:
# Prepare the notebook for use of ModelFlow 

# Jupyter magic command to improve the display of charts in the Notebook
%matplotlib inline

# Import pandas 
import pandas as pd

# Import the model class from the modelclass module 
from modelclass import model 

# functions that improve rendering of ModelFlow outputs
model.widescreen()
model.scroll_off();

In [ ]:
# set default precision 
pd.set_option("display.precision", 2)

In [ ]:
# For display reasons 
latex = 1


## Load the existing model, data and descriptions 
The file `pak.pcim` contains a dump of model equations, dataframe, simulation options and variable descriptions for the World Bank climate aware model for Pakistan described in {cite:t}`burns_climate_2021`. 

The code below:

 - Loads the model and simulates it using the `DataFrame` stored in the `pcim` file to establish a baseline.
 - Creates a `DataFrame` that is a copy (from 2020) of the active solution in `mpak` and changes the tax rate to 30 USD/Ton for carbon emissions from coal, oil and natural gas.
 - Runs a simulations with these new carbon taxes. 
 
The results from this simulation will be used below to explore the attribution functionality of ModelFlow.
  
  
{ref}`Single equation Impact Decomposition <impactsingle>`
  

In [ ]:
mpak,baseline = model.modelload('../models/pak.pcim',alfa=0.7,run=1,relconv=0.0000000001 ,keep='Business as Usual')
alternative  =  baseline.upd("<2020 2100> PAKGGREVCO2CER PAKGGREVCO2GER PAKGGREVCO2OER = 30")

In [ ]:
#simulate the model
result = mpak(alternative,2025,2100,keep='Nominal carbon tax of 30 USD',
              ljit=False,         # do not compile the model (default value for this option)
              nfirst=800,
              maxiteration=100   # if no convergence after 100 iterations - stop
             ) # simulates the model 


## The mathematics of decomposition

At its root the idea of attribution is to take the total derivative of the model to identify the sensitivity of the equation of interest to changes elsewhere in the model and then combine that with the changes in other variables. 

Take a variable y that is a function of two other variables a and b.  In the model, the relationship might be written as:


$y = f(a,b)$

If there are two sets of results designated with a subscript 0 and 1, these can be written as: 

\begin{eqnarray}
y_0 = f(a_0,b_0)\\
y_1 = f(a_1,b_1)
\end{eqnarray}

If the change in the three variables is specified as $\Delta y, \Delta a, \Delta b$, the total derivative of y can be written as:


$\Delta y = \underbrace{\Delta a \dfrac{\partial {f}}{\partial{a}}(a,b)}_{\Omega a} + 
\underbrace{\Delta b \dfrac{\partial {f}}{\partial{b}}(a,b)}_{\Omega b}+Residual$

The first expression can be called $\Omega_a$ or the contribution of changes in a to changes in y, and the second $\Omega_b$,  or the contribution of changes in b to changes in y.  


`ModelFlow` performs a numerical approximation of $\Omega_a$ and $\Omega_b$ by performing two runs of the $f()$:

\begin{eqnarray}  
y_0&=&f(a_{0},b_{0}) \\
y_1&=&f(a_0+\Delta a,b_{0}+ \Delta b)
\end{eqnarray}

and calculates $\Omega_a$ and $\Omega_b$ as:

\begin{eqnarray}  
\Omega a&=&f(a_1,b_1 )-f(a_1-\Delta a,b_1) \\
\Omega b&=&f(a_1,b_1 )-f(a_1,b_1-\Delta  b)
\end{eqnarray}



And: 

\begin{eqnarray}
residual = \Omega a + \Omega b -(y_1 - y_0) 
\end{eqnarray}

If the model is fairly linear, the residual will be small. 



## Model-level decomposition or  single equation decomposition?

Above, the relationship between y, a, and b was summarized by the function f(). 

$f(a,b)$ could represent **a single equation** in the model or it could represent **the entire model**. 

In the **single equation** mode, $\Delta a$ and $\Delta b$ would be treated as exogenous variables in the attribution calculation as they are both on the right hand side of the equation (i.e. exogenous to this equation -- even if they might be endogenous variables in some other equation). Here results will show the direct impact of changes in the RHS variable(s) on the LHS variable.

When analyzing the total derivative for the **entire model** instance, $a$ and $b$ will be purely exogenous variables. In this case, the decomposition shows the cumulative effect -- potentially operating through multiple channels of a change in the exogenous (or exogenized variables) on different endogenous variables in the model. Say we are looking at inflation, an exogenous change in wages would influence prices directly through higher costs of production and indirectly by inducing higher demand.  The  model-level decompoistion will return the sum of the two or more influences.

Assume the simple equation example such that  $a$ and $b$ are simple variables. When $\Delta y$, $\Delta a$ and $\Delta b$ reflect the difference across scenarios (say the value of the three variables in `.lastdf` less the value in `.basedf`) then;

$\Omega_a$, $\Omega_b$ are the absolute contribution of a and b to the change in y, and 
$100*\bigg[\cfrac{\Omega_a}{\Delta y}\bigg]$ is the share of the change in y explained by expressed as a percent and  $100*\bigg[\cfrac{\Omega_b}{\Delta y}\bigg]$ is the share of the change in y explained by b expressed as a percent.

If $\Delta y$, $\Delta a$ and $\Delta b$ are the changes over time ($\Delta y_t=y_t-y_{t-1}$), then $\Omega_a$, $\Omega_b$ are the contributions of a and b to the rate of growth of y, while $100*\bigg[\cfrac{\Omega_a}{\Delta y_{t-1}}\bigg]$  $100*\bigg[\cfrac{\Omega_b}{\Delta y_{t-1}}\bigg]$ are are the contributions of a and b to the rate of growth of y.


:::{index} single: Impact Decomposition; Single equation 
:name: impactsingle
:::





## Decomposing the source of changes to a single endogenous variable

The `ModelFlow` method `.dekomp()` is used to calculate the contribution of RHS variables to the change in an endogenous (LHS) variable. 

This method takes advantage of the fact that the model object stores the initial and most recent simulation result in two dataframes called `.basedf` and `.lastdf`, as well as all of the equations of the model. 



The `dekomp()` method calculates the contribution to changes in the level of the dependent variable in a given equation. It does not calculate what caused the changes to the RHS variables.

In the example below, the contribution to the change in Total emissions is decomposed into the contribution from each of three sources in the model, the consumption of Crude Oil, Natural Gas and Coal.  As the equation for total emissions is just the sum of the three this is a fairly trivial decomposition, but it provides an easily understood illustration of the process at work.

Note that, initially some carbon taxes were negative because the associated energy products benefited from some sort of subsidy.  As a result, although each carbon tax is set to 30 in the simulation that was run earlier, the change in the levels of the Carbon tax is different across carbon taxes, with the increase in the net taxation on the carbon emissions from natural gas being particularly large.  


In [ ]:
print("Change in carbon taxes:") 
with mpak.set_smpl(2023,2030):
    print(mpak['PAKGGREVCO2CER PAKGGREVCO2GER PAKGGREVCO2OER'].dif.rename().df)

:::{only} latex
latexcommand \begin{samepage}
:::

In [ ]:
dekomp_result = mpak.PAKCCEMISCO2TKN.dekomp(start=2024,end=2027);

:::{only} latex
latexcommand \end{samepage}
:::

The above results from the call to `.dekomp()` are presented in several sections.

|Section|Table|Contents|
|:--|:--|:--|
|**The first section**| |the normalized formula of the RHS variable `PAKCCEMISCO2TKN`|
|**The second section**| |Shows the changes in level terms. |
|| diff_level |First by showing the results of the simulation **base**, then the previous level **last**, then the difference and then the difference expressed as a percent|
|| att_level |This is followed by a table showing the contribution of the changes in every LHS variable to the observed change in the dependent variable.|
|**The third section**| att_pct |Shows the same results for the RHS variables, but expressed as a percent of the total change in the dependent variable. |
|**The fourth section**|| Shows the same results but for the change in the growth rate of the dependent variable.|
|| diff_growth |The first table shows the post-shock growth rate of the dependent variable from the `.lastdf` dataframe, followed by the pre-shock growth rate and the difference in the growth rates. |
|| att_growth |The second table of this section shows the contribution to the change in the growth rate from each RHS variable. |


The object returned by `.dekomp()` is a [namedtuple](https://realpython.com/python-namedtuple/) that contains each of these tables which can then be referred to later.

The code below extracts the different sub-components of the `.dekomp()` results and displays them individually.



:::{only} latex
latexcommand \begin{samepage}
:::

In [ ]:
# Loop over the elements in the result of dekomp. 
# a named tuple can be used both as a straight tuple and the elements
# can be accessed through the field name. 

with pd.option_context('display.float_format', '{:.2f}'.format):
    for f,df in zip(dekomp_result._fields,dekomp_result):
        display(f)
        display(df)
    

:::{only} latex
latexcommand \end{samepage}
:::

## A more complex example

The above decomposition is fairly straight forward because the decomposed equation is a simple identity, where Total Emissions are just the sum of its three component parts: Total Carbon emissions = Emissions from Oil+  Emissions from Coal + Emissions from Natural Gas.

The following single-equation decomposition looks to the impact of the same shock (introduction of a carbon tax) on a different variable (inflation).  The inflation equation is more complex and has more direct causal variables, so the decomposition is more interesting.

Recall the inflation equation is given by the `.frml` method for its normalized version and `.eviews` for its original specification.  The equation for the consumer price level (PAKNECONPRVTXN) was originally specified in eviews as:

In [ ]:
mpak['PAKNECONPRVTXN'].eviews

The normalized equation is given by `mpak['PAKNECONPRVTXN'].frml`.  

Note in the Pakistan model, consumer inflation is derived as a constant elasticity of transformation (CET) aggregation of the price of energy goods(PAKNECONENGYXN) and non-energy goods (PAKNECONOTHRXN).

In [ ]:
mpak['PAKNECONPRVTXN'].frml

Note further the normalized equation is solving for the **level** of the price deflator -- not inflation which is the rate of growth of this index.

Because the equation solves for the level of the price deflator, the decomposition show the contributions of each explanatory variable to the increase in the price level (not that of the inflation rate). However, the 4th table is showing the impacts on the rate of growth of the price level -- i.e. the level of inflation.

:::{only} latex
latexcommand \begin{samepage}
:::

In [ ]:
mpak['PAKNECONPRVTXN'].dekomp(start=2024,end=2027);

:::{only} latex
latexcommand \end{samepage}
:::

Interestingly only 23% of the increase in the price level each period is due to the direct channel (the impact on the price of energy consumed by households), the bulk of the increase comes indirectly through other prices.  Indeed as time progresses this share rises from 77% in the first year of the price change (2020) to 83% by 2024.

### Non-energy prices

Below is the formula for nonenergy consumer prices and their decomposition. This equation is written out as a more standard augmented-phillips-curve type inflation equation reflecting changes in the cost of local goods production (PAKNYGDPFCSTXN), Government taxes on goods and services (PAKGGREVGNFSXN), the price of imports (PAKNEIMPGNGSXN) and the influence of the economic cycle (PAKNYGDPGAP_) on the price level.

In [ ]:
mpak['PAKNECONOTHRXN'].eviews

In [ ]:
mpak['PAKNECONOTHRXN'].dekomp(start=2025,end=2029);

These results indicate that much of the initial impact on prices is coming from the increase in the price of imported goods (which includes a large fuel component). As time progresses, the imported inflation component declines (because fuel and import prices are no longer rising) and the lagged consumption price dominates (the level this period is basically determined by the price level in the previous period) .  Other factors such as the cost of domestically produced goods play a larger role and the net impact of imported prices (the total of the contemporaneous and lagged value) approaches zero. Cyclical pressure are initially adding to inflation before declining and eventually turning negative. 

:::{index} single: Impact Decomposition; Single equation,  .get_attr() more output transformations 
:::

## The `get_att()` method provides more control over the outputs of `.dekomp()`

Following a call to the `.dekomp()` method, the `.get_att()` method provides a range of mechanisms that allow the results to be displayed in different ways.




### The default display of `.get_att()` 

By default `.get_att()` displays the share contributions of RHS variables to the total change in the LHS variable.  The start= and end= options allow the period for which results are displayed to be restricted.



In [ ]:

mpak.PAKNECONPRVTKN.get_att(start=2025,end=2035)

:::{image} Attrib-get_att.png
    :alt: get attribution 
    :class: bg-primary mb-1
    :width: 100%
    :align: center

:::    

:::{index} single: Impact Decomposition; Single equation, impact accumulated across lags 
:::

### Options: Lag=True/False 

Because the decomposition of the equation is based on the normalized (levelized) version of the equation, for equations initially written as growth rates or ECMs many variables will occur several times in the attribution table with both the contribution of the current value of the variable and those of any lagged versions that appear in the normalized equation.

The `Lag=False` option changes the default behavior of `.get_att()` and aggregates the contributions of different lags.

By aggregating the lags, the net effect of changes in the variables can be more easily determined. Below it is clearer that the initial impact of higher import prices drove most of the inflation response.  In subsequent periods, import prices were stable or even falling so most of the contribution to the change in the level of other goods inflation was from the lagged dependent variable and the changed state of the economic cycle (the Gap variable which initially was adding to inflationary pressures eventually subtracts from inflation as the economy slows).


In [ ]:
mpak.PAKNECONOTHRXN.get_att(lag=False,start=2025,end=2035)

:::{image} Attrib-get_att.png
    :alt: Menu to start notebooks in subfolders
    :class: bg-primary mb-1
    :width: 100%
    :align: center

:::    

:::{index} single: Impact Decomposition; Single equation, output as "growth/pct/Level"
:::

:::{index} single: Impact Decomposition; Single equation, cutout threshold 
:::

### Options: Type="growth/pct/Level"

The option `Type` controls which of the tables generated by `dekomp()` is displayed. The contributions of RHS variables to the level of the dependent variable (`=level`), the share of the observed change in the RHS variable attributable to each dependent variable (`=pct`), and the change in the growth rate (`=growth`)of the LHS available attributable to the changes in the growth rate of each RHS variable.

### Options: threshold=xx"

The `threshold=` option will suppress from the output those variables whose contribution is less than the stated threshold.

In the example below, lags are suppressed, and only contributions to the growth rate of variables whose largest contribution was more than $\pm$0.1 percent are displayed.  The dropped variables influence is aggregated and displayed in a row labeled **small**.

#### Options: bare=True/False

If bare is set to `False` then the values of the LHS variable in the `basedf` and `lastdf` dataframes and their difference are also displayed. By default this option is `True` which suppresses the display.



In [ ]:
mpak.PAKNECONPRVTKN.get_att(lag=False,type='growth',bare=False,threshold=0.1,start=2020,end=2024)

:::{image} Attrib-get_att-bare.png
    :alt: Menu to start notebooks in subfolders
    :class: bg-primary mb-1
    :width: 70%
    :align: center

:::    

:::{index} single: Scenarios; Impact Decomposition examples
:::
:::{index} single: Impact Decomposition; examples
:::

### Several examples
Here the default type (pct) is displayed and the threshold is set to 10, so only variables whose aggregate impact was more than 10 percent of the total in one or more of the displayed years are shown.


In [ ]:
mpak.PAKNECONPRVTKN.get_att(lag=False,threshold=10,start=2025,end=2035)

:::{image} Attrib-get_att-pct.png
    :alt: Menu to start notebooks in subfolders
    :class: bg-primary mb-1
    :width: 100%
    :align: center

:::  


The three examples below show the impact on real consumption of the changes induced on its LHS variables as changes in percent level and growth, with the threshold set to focus only on the main channels.

In [ ]:
mpak.PAKNECONPRVTKN.get_att(lag=False,threshold=10,bare=False,start=2025,end=2029);
mpak.PAKNECONPRVTKN.get_att(lag=False,threshold=10,type='level',bare=False,start=2025,end=2029);
mpak.PAKNECONPRVTKN.get_att(lag=False,threshold=0.1,type='growth',bare=False,start=2025,end=2029);

:::{image} Attrib-get_att-multi.png
    :alt: Menu to start notebooks in subfolders
    :class: bg-primary mb-1
    :width: 100%
    :align: center

:::    

:::{index} single: Impact Decomposition; Single equation, charts , .dekomp_plot()
:::


### Displaying `.dekomp()` results graphically


The `dekomp_plot` method allows singe-equation decompositions to be displayed graphically. Below in the initial periods, the import price, and cost-push factors dominate, but as the model equilibrates the lagged level of the price deflator explains virtually all of the movement in the level of the price.


In [ ]:
fig=mpak.dekomp_plot('PAKNECONOTHRXN',pct=True,rename=True,threshold=.01,lag=False); #decomp of the change in the level

:::{index} single: Impact Decomposition; Single equation, charts
:::


In the following example the change in the level of the dependent variable is displayed for a restricted time period.  Here the distinction between the initial impulse (import prices) and the lagged effect of past prices is very evident.

In [ ]:
with mpak.set_smpl(2020,2030):
    fig=mpak.dekomp_plot('PAKNECONOTHRXN',pct=False,rename=True,threshold=.005,lag=False); #decomp of the change in the level

:::{index} single: Impact Decomposition; Single equation, time dimension 
:::



### the time_att option

The above displays focused on the difference between the values in the two dataframes `basedf` and `lastdf`.

By setting he time_att option to True, `get_att()` displays the contribution of changes in the levels of the RHS variables between t and t-1,in explaining the changes in the LHS variable between t and t-1 with all data pulled from the same `.lastdf` datafame.

:::{note}
With the `time_att` option set **only the .lastdf dataframe** is used.   The comparison is not .basedf vs .lastdf but the influence of last year's changes on the level of this year's variable. The attribution is calculated by lagging each right hand side variable one year and recalculating the equation.
:::

In [ ]:
mpak.PAKCCEMISCO2TKN .get_att(time_att= True,type='level',bare=0);

:::{image} get_att.png
    :alt: Menu to start notebooks in subfolders
    :class: bg-primary mb-1
    :width: 70%
    :align: center
:::    

In [ ]:
help(mpak.get_att)

In [ ]:
help(mpak.dekomp_plot)

:::{index} single: Impact Decomposition; Single equation, Trace preceding variables 
:::

## Trace and decomposition combined

The `.tracepre()` method can combine the graphical representation of the `tracepre()` method described in the previous chapter and the tabular results from `dekomp()`.  

This is implicit in the standard call to `.tracepre()`where the thickness of the lines is derived from the empirical importance of the changes in each LHS variable in determining the change in the RHS variable.  


In [ ]:
mpak.PAKNECONPRVTKN.tracepre(png=latex,size=(2,4));

## Tabular output from tracepre()

The results for `.tracepre` can be displayed in a number of ways and the results can be saved as pictures. 

|||
|:--|:--|
|**up = xx**|determines how many levels of parents to include|
|**showdata\|sd=True**|Causes the tables of attribution for each displayed variable to be displayed|
|**showdata\|sd=\<'pattern of variable names'>**|will include a table of values for each variable matching the pattern (including wildcharts|
|**attshow\|ats = True**| adds in the contribution of each to the total change|
|**growthshow\|gs = True**| will include a table of growth for each variable|
|**HR = True**| will reorient the dependency graph|
|**filter=\<xx>**|restrict outputs to variables that explain at least xx% of the change in the level of dependent variable |
|**browser = True**|Opens a browser with the resulting dependency graph - useful for zooming on a big graph or table|
|**pgn = True**|will display as a png picture
|**svg = True**|will display as a svg picture which can be zoomed
|**pdf = True**|will display as a pdf picture
|**eps = True**|will create a eps file (a latex format)
|**saveas = \<a file name without extension>**|will save the picture wit the filename with an added extension reflection the picture type     
    

In [ ]:
with mpak.set_smpl(2020,2030):
    mpak.PAKNECONOTHRXN.tracepre(filter=5.0,HR=False,showdata= True,attshow=True,per=2020,png=latex)

The big difference with this representation is the contributions of both the direct that directly impact the LHS variable (as well as the influence of those variables one or two steps up the causal chain) can be traced.

Below the same command as above but we specify that we want to go up two levels in the causal chain.

In [ ]:
with mpak.set_smpl(2020,2030):
    mpak.PAKNECONOTHRXN.tracepre(up=2,filter=5,HR=False,sd= True,ats=True,png=latex)

In [ ]:
with mpak.set_smpl(2020,2023):
    mpak.PAKNECONPRVTKN.tracepre(sd='*lcn',filter=10,HR=1,ats=1,up=2
    ,growthshow=1,png=latex)

As indicated by the error message the filter is too fine, and has eliminated all variables from the output.  Below the same command without the filter option.

In [ ]:
with mpak.set_smpl(2020,2023):
    mpak.PAKNECONPRVTKN.tracepre(sd='*lcn',HR=1,ats=1,up=2
    ,growthshow=1,png=latex)

## Chart of the contributions over time 

:::{index} single: Impact Decomposition; Single equation, Waterfall graphs
:::

In [ ]:


with mpak.set_smpl(2020,2030):
    mpak.dekomp_plot('PAKNYGDPFCSTXN',threshold=5);  # gives a waterfall of contributions

## Chart of the contributions for one year
It can be useful to visualize the attribution as a waterfall chart for a single year

In [ ]:
mpak.dekomp_plot_per('PAKNYGDPFCSTXN',per=2027,threshold=5)  # gives a waterfall of contributions

In [ ]:
mpak.dekomp_plot_per('PAKNYGDPFCSTXN',per=2027,threshold=5)  # gives a waterfall of contributions

## Sorted waterfall of contributions

In [ ]:
mpak.dekomp_plot_per('PAKNYGDPFCSTXN',per=2029,threshold=5,sort=True)  # gives a waterfall of contributions

:::{index} single: Impact Decomposition; Whole model .totdif() method
:name: impactmodel
:::

:::{index} single: Impact Decomposition; Find all exogenous shocks .exodif() method
:name: exodif
:::


## Impacts at the model level: the `.totdif()` method 

The method `.totdif()` returns an instance of the totdif class, which provides a number of methods and properties to explore decomposition at the model level.

It works by solving the model numerous times, each time changing one of the right hand side variables and calculating the impact on all dependent variables. By default it uses the values from the `.lastdf` `DataFrame` as the shock values and the values in `.basedf` as the initial values. Separate simulations are run for every exogenous (or exogenized) variables that have changed between the two `DataFrame`s. 

For advanced users the RHS variables can be grouped into user defined blocks, which in cases where there are many changes can help identify the main causal pathways.

### The `.exo_dif()` method 
The `.exodif()` method displays only the exogenous variables that have changed between the two `DataFrame`s (the shock). Exogenous variables whose results have not changed are omitted. It determines which of the exogenous variables have changed between `.lastdf`and `.basedf` and then returns a `DataFrame` with the changes in the values. 

In this case the `DataFrame` contains the effect of updating the $CO^2$ tax to 30 for coal, gas and oil. `.exo_dif()` is automatically called by the `.totdif()` method but can also be called directly b y the user. 

In [ ]:
mpak.exodif()

### The `.totdif()` command calculates the contribution of each changed variable to the changes in a specified LHS variable
This involves solving the model a number of times, so can take some time. How long it takes to execute will depend on the computer, the model and the number of changes made.  In this instance the `.totaldif` takes between 2 and 5 seconds depending on computer. 

In [ ]:
totdekomp = mpak.totdif() # Calculate the total derivative½s of all equations in the model.

:::{index} single: Impact Decomposition; Whole model as charts 
:::

### The method `.explain_all()` presents the results graphically

In the example below, the relative importance of the three shocked carbon taxes on the change in real GDP are presented.

In [ ]:
showvar = 'PAKNYGDPMKTPKN'
totdekomp.explain_all(showvar,kind='area',use='growth',stacked=True,
                      title="Contributions of different carbon taxes to Real GDP growth") ;

In [ ]:
help(totdekomp.explain_all)

### Many variables

If many variables are passed to explain_all then separate graphs will be created for each.

In [ ]:
showvar = 'PAKNYGDPMKTPKN PAKCCEMISCO2CKN PAKCCEMISCO2OKN PAKCCEMISCO2GKN PAKGGREVTOTLCN'

totdekomp.explain_all(showvar,kind='area',stacked=True,title="Contributions of different carbon taxes to Real GDP,growth") ;


:::{index} single: Impact Decomposition; Whole model - waterfall 
:::

### Similarly the impacts on different variables for one year can be shown

In [ ]:
showvar = 'PAKNYGDPMKTPKN PAKNECONPRVTXN'

totdekomp.explain_per(showvar,per=2028,ysize=8,title='Decomposition, level=2023')

:::{index} single: Impact Decomposition; Whole model - interactive widget
:::


### Or an interactive widgets can be generated
This allows the user to select the specific variable of interest and what to display: 


:::{note}
If this is read in a manual the widget is not live. 

In a notebook the selection widgets are live. 
:::


In [ ]:
mpak.get_att_gui(var='PAKGGREVTOTLCN',ysize=7)

:::{image} get_att_gui.png
    :alt: Menu to start notebooks in subfolders
    :class: bg-primary mb-1
    :width: 70%
    :align: center
:::

:::{index} single: Impact Decomposition; Whole model - last year
:::

### Decomposition of the last year 

In [ ]:
showvar = 'PAKNYGDPMKTPKN'
totdekomp.explain_last(showvar,ysize=8,title='Decomposition last period, level')

:::{index} single: Impact Decomposition; Whole model - accumulated effects
:::

### Decomposition of accumulated effects 

In [ ]:
totdekomp.explain_sum(showvar,ysize=8,title="Decomposition, sum over all periods,level")

## More advanced model attribution

For some  simulations the number of changed exogenous variables can be large. Using a dictionary to contain the experiments allows us to manage multiple scenarios and multiple outputs. 

Using this approach, if there are many simulations, data can be filtered in order to look only at the variables with an impact above a certain threshold. 



:::{index} single: Impact Decomposition; Whole model - grouping variables 
:::


### Grouping variables
If many exogenous variables were shocked, exploring impacts may be made easier by aggregating the impacts of some groups or sub-groups of variables. Grouping  variables allows the user to explore the results in a more flexible way slicing and dicing the impact along different dimensions. 

In the example below, the impacts of changing the carbon tax on gas and oil tax are grouped together (aggregated) and the impact of the coal tax is displayed separately. 

In [ ]:
shocks = {'gas and Oil':['PAKGGREVCO2OER', 'PAKGGREVCO2GER'],'Coal':['PAKGGREVCO2CER']}
totdekomp_group = mpak.totdif(experiments = shocks) # Calculate the total derivative½s of all equations in the model.

In [ ]:
showvar = 'PAKNYGDPMKTPKN'
totdekomp_group.explain_all(showvar,kind='area',stacked=True,title='GDP impact of coal and non-coal carbon taxes');    

While this is a fairly simple example, the grouping mechanism allows us to focus our attention on one factor (the coal price in this instance).  

Here, even though the coal tax was increased by the most (in the baseline it was subsidized), it had a relatively small share in total energy production, so its GDP impact was relatively small.

### Single equation attribution chart 
The results can be visualized in different ways. 

In [ ]:
mpak.dekomp_plot_per('PAKNYGDPMKTPKN',
                     per=2025,           # Period to be displayed
                     pct=False,          # Do not show differences as percent changes
                     rename=True,        # Use the long-form vs mnemonic description of variable
                     sort=True,          # 
                     threshold =200000,
                     ysize=7            # Size of y axis in inches   
                     )

### Decomposition of changes over time 

:::{index} single: Impact Decomposition; Single equation - impact over time 
:::


A classic query is to understand what is driving changes over time.  The `time_att=True` option quantifies the impact of changes over time in the LHS variables on changes in the dependent variable over time using data from the `lastdf` `DataFrame`.  


In [ ]:
with mpak.set_smpl(2020,2024):
    mpak['PAKNYGDPMKTPKN'].dekomp(time_att=True)

In [ ]:
mpak.dekomp_plot('PAKNYGDPMKTPKN',pct=0,rename=1,sort=1,threshold =0,time_att = True);